# Solve TSP problem using Genetic Algorithm

## Import libraries

In [45]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt

## Model Gen and Chromosome

In [46]:
class City: # Gen in GA
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance

    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

In [49]:
class Route: # Chromosome in GA
    def __init__(self, cities : list):
        self.cities = cities
        self.fitness = 0
        self.distance = 0

    def calculate_distance(self):
        if self.distance == 0:
            pathDistance = 0
            for i in range(0, len(self.cities)):
                fromCity = self.cities[i]
                toCity = None
                if i + 1 < len(self.cities):
                    toCity = self.cities[i + 1]
                else:
                    toCity = self.cities[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance

    def calculate_fitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.calculate_distance())
        return self.fitness

## GA functions

In [50]:
def create_route(cityList):
    route = random.sample(cityList, len(cityList))
    return route

In [51]:
def initial_population(popSize, cityList):
    population = []

    for i in range(0, popSize):
        population.append(create_route(cityList))
    return population

In [52]:
def rank_routes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Route(population[i]).calculate_fitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

In [53]:
def selection(popRanked, eliteSize, population):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()

    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for _ in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break

    mating_pool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        mating_pool.append(population[index])
    return mating_pool

In [54]:

def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []

    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])

    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

In [55]:
def breed_population(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])

    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

In [56]:
def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))

            city1 = individual[swapped]
            city2 = individual[swapWith]

            individual[swapped] = city2
            individual[swapWith] = city1
    return individual

In [57]:
def mutate_population(population, mutationRate):
    mutatedPop = []

    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

In [58]:
def next_generation(currentGen, eliteSize, mutationRate):
    popRanked = rank_routes(currentGen)
    mating_pool = selection(popRanked, eliteSize, currentGen)
    children = breed_population(mating_pool, eliteSize)
    nextGeneration = mutate_population(children, mutationRate)
    return nextGeneration

## Genetic algorithm main functions

In [59]:
def genetic_algorithm(population, population_size, eliteSize, mutationRate, generations):
    pop = initial_population(population_size, population)
    print("Initial distance: " + str(1 / rank_routes(pop)[0][1]))

    for i in range(0, generations):
        pop = next_generation(pop, eliteSize, mutationRate)

    print("Final distance: " + str(1 / rank_routes(pop)[0][1]))
    bestRouteIndex = rank_routes(pop)[0][0]
    bestRoute = pop[bestRouteIndex]
    return bestRoute

In [60]:
def genetic_algorithm_plot(population, popSize, eliteSize, mutationRate, generations):
    pop = initial_population(popSize, population)
    progress = []
    progress.append(1 / rank_routes(pop)[0][1])

    for i in range(0, generations):
        pop = next_generation(pop, eliteSize, mutationRate)
        progress.append(1 / rank_routes(pop)[0][1])

    plt.plot(progress)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    plt.show()

## Read dataset or generate random data

In [66]:
cityList = []
with open('dataset.txt', 'r') as f:
    for line in f:
        id, x, y = line[:-1].split(' ')
        cityList.append(City(float(x), float(y)))

In [ ]:
cityList = []

for i in range(0,25):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

## Run alogrithm

In [ ]:
genetic_algorithm(population=cityList, population_size=100, eliteSize=20, mutationRate=0.01, generations=100)

In [ ]:
genetic_algorithm_plot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=100)